## **Code By Shoaib Shoukat**

## **Import** **Libraries**

In [18]:
import requests
import pandas as pd
from datetime import datetime, timedelta

## Crypto Want To Scrap

In [19]:
# Currencies Name
cryptos = [
    {"name": "Bitcoin", "symbol": "bitcoin"},
    {"name": "Ethereum", "symbol": "ethereum"}
]

## Api To Access Coins Data

In [20]:
# CoinGecko API endpoints
current_price_url = "https://api.coingecko.com/api/v3/simple/price"
historical_data_url = "https://api.coingecko.com/api/v3/coins/{id}/market_chart/range"


## Function to get the current price

In [21]:
def get_current_price(crypto_symbol):
    params = {
        "ids": crypto_symbol,
        "vs_currencies": "usd"
    }
    response = requests.get(current_price_url, params=params)
    data = response.json()
    return data.get(crypto_symbol, {}).get("usd", None)

## # Function to fetch historical data for the last month

In [22]:
def get_historical_data(crypto_symbol):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30)

    params = {
        "vs_currency": "usd",
        "from": int(start_date.timestamp()),
        "to": int(end_date.timestamp())
    }
    response = requests.get(historical_data_url.format(id=crypto_symbol), params=params)
    data = response.json()

    prices = data.get("prices", [])
    # Parse the data into a readable format
    historical_data = []
    for p in prices:
        timestamp = datetime.utcfromtimestamp(p[0] / 1000)
        historical_data.append({
            "date": timestamp.strftime('%Y-%m-%d'),
            "time": timestamp.strftime('%H:%M:%S'),
            "historical_price": p[1]
        })
    return historical_data

# Collect data for Bitcoin and Ethereum
all_crypto_data = []
for crypto in cryptos:
    print(f"Fetching data for {crypto['name']}...")
    current_price = get_current_price(crypto["symbol"])
    historical_data = get_historical_data(crypto["symbol"])

    # Combine data with the current price, name, and symbol
    for record in historical_data:
        record["current_price"] = current_price
        record["name"] = crypto["name"]
        record["symbol"] = crypto["symbol"]

    all_crypto_data.extend(historical_data)



Fetching data for Bitcoin...
Fetching data for Ethereum...


## Convert And Save The Data

In [23]:
# Convert the collected data into a DataFrame
df = pd.DataFrame(all_crypto_data)

# Ensure columns are ordered properly
df = df[["date", "time", 'name', "symbol", "current_price", "historical_price"]]

# Display the first few rows
print(df.head())

# Save the DataFrame to a CSV file
df.to_csv("bitcoin_ethereum_monthly_data.csv", index=False)
print("Data saved to bitcoin_ethereum_monthly_data.csv")


         date      time     name   symbol  current_price  historical_price
0  2024-11-01  15:14:56  Bitcoin  bitcoin        97255.0      70923.828013
1  2024-11-01  16:06:16  Bitcoin  bitcoin        97255.0      69345.353725
2  2024-11-01  17:02:35  Bitcoin  bitcoin        97255.0      69844.909336
3  2024-11-01  18:09:23  Bitcoin  bitcoin        97255.0      69503.663605
4  2024-11-01  19:21:28  Bitcoin  bitcoin        97255.0      69020.890997
Data saved to bitcoin_ethereum_monthly_data.csv
